<a href="https://colab.research.google.com/github/karloxkronfeld/Finanzas/blob/main/SL_TP_vela_doji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd

df = yf.download("^GSPC", period="10y", interval="1d")

def detectar_doji(open, close, high, low):
    body_size = abs(close - open)
    shadow_size = min(abs(open - low), abs(close - low), abs(open - high), abs(close - high))
    return body_size <= shadow_size * 0.1

df['Doji'] = [detectar_doji(row['Open'], row['Close'], row['High'], row['Low']) for _, row in df.iterrows()]

# Parámetros de stop loss y take profit (en porcentaje)
stop_loss_pct = 1.0  # 1% de pérdida máxima permitida
take_profit_pct = 2.0  # 2% de ganancia objetivo

# Variables para realizar el seguimiento de las operaciones
operaciones_abiertas = False
precio_compra = 0.0
stop_loss = 0.0
take_profit = 0.0
operaciones_ganadoras = 0
operaciones_perdedoras = 0

for _, row in df.iterrows():
    if row['Doji']:
        if not operaciones_abiertas:
            operaciones_abiertas = True
            precio_compra = row['Close']
            stop_loss = precio_compra - (precio_compra * stop_loss_pct / 100)
            take_profit = precio_compra + (precio_compra * take_profit_pct / 100)
    else:
        if operaciones_abiertas:
            if row['Low'] <= stop_loss:
                operaciones_perdedoras += 1
                operaciones_abiertas = False
            elif row['High'] >= take_profit:
                operaciones_ganadoras += 1
                operaciones_abiertas = False

tasa_aciertos = operaciones_ganadoras / (operaciones_ganadoras + operaciones_perdedoras)
rendimiento_acumulado = ((operaciones_ganadoras * take_profit) - (operaciones_perdedoras * stop_loss)) / (operaciones_ganadoras + operaciones_perdedoras)

print("Resultados:")
print("Operaciones ganadoras:", operaciones_ganadoras)
print("Operaciones perdedoras:", operaciones_perdedoras)
print("Tasa de aciertos:", tasa_aciertos)
print("Rendimiento acumulado:", rendimiento_acumulado)


[*********************100%***********************]  1 of 1 completed
Resultados:
Operaciones ganadoras: 23
Operaciones perdedoras: 39
Tasa de aciertos: 0.3709677419354839
Rendimiento acumulado: -1009.1341788999498
